# PART 1

In [27]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from matplotlib import pyplot as plt

background = cv2.imread('Malibu.jpg')
#plt.imshow(background)
#cv2.imshow('Background Image Window', background)
#cv2.waitKey(0)

background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background, (int(background_width*ratio),360))
print(background.shape)

imageslist = []
i = 0
while 1: # loop through cat frames
    image=cv2.imread('cat/cat_'+ str(i) + '.png')
    if image is None:
        break
    i = i+1
    image_g_channel = image[:, :,1]
    image_r_channel = image[:, :,0]
    foreground = np.logical_or(image_g_channel < 180 , image_r_channel > 150)
    nonzero_x , nonzero_y = np.nonzero(foreground)
    nonzerocatvalues = image[nonzero_x, nonzero_y, : ]
    new_frame = background.copy()
    new_frame[nonzero_x, nonzero_y, :] = nonzerocatvalues
    new_frame = new_frame[:,:,[2, 1, 0]]
    imageslist.append(new_frame)
clip = mpy.ImageSequenceClip( imageslist , fps = 25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration( clip.duration )
clip = clip.set_audio(audioclip=audio )
clip.write_videofile('part1_video.mp4', codec='libx264')

    

(360, 926, 3)


chunk:   0%|                                 | 0/159 [00:00<?, ?it/s, now=None]

Moviepy - Building video part1_video.mp4.
MoviePy - Writing audio in part1_videoTEMP_MPY_wvf_snd.mp3


t:   0%|                                     | 0/181 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part1_video.mp4



Moviepy - Done !
Moviepy - video ready part1_video.mp4


# PART2

In [28]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from matplotlib import pyplot as plt

background = cv2.imread('Malibu.jpg')
background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background, (int(background_width*ratio),360))
print(background.shape)

imageslist = []
i = 0
while 1:
    image=cv2.imread('cat/cat_'+ str(i) + '.png')
    if image is None:
        break
    i = i+1
    
    #place the reflected cat to the right side
    mirror_im = cv2.flip(image,1)
    green_im = np.zeros((360,286,3), np.uint8)
    green_im[:,0:286]= (0,255,0)
    mirror_im = cv2.hconcat([green_im, mirror_im])
    
    #at first tries i thinked of shifting right but instead of this i choose concatenating with green image
        #M = np.float32([[1,0,286],[0,1,0]])
        #rows,cols,value = mirror_im.shape
        #mirror_im = cv2.warpAffine(mirror_im,M,(cols,rows))
    
    with_myself = mirror_im.copy()
    with_myself[0:image.shape[0],0:image.shape[1]-110,:] = image[0:image.shape[0],0:image.shape[1]-110,:]
    #i left 110 more pixels to right cat to see its head shaking better , otherwise it was cropping by orginal
    #cat figures
    image = with_myself
    #same for the rest
    image_g_channel = image[:, :,1]
    image_r_channel = image[:, :,0]
    foreground = np.logical_or(image_g_channel < 180 , image_r_channel > 150)
    nonzero_x , nonzero_y = np.nonzero(foreground)
    nonzerocatvalues = image[nonzero_x, nonzero_y, : ]
    new_frame = background.copy()
    new_frame[nonzero_x, nonzero_y, :] = nonzerocatvalues
    new_frame = new_frame[:,:,[2, 1, 0]]
    imageslist.append(new_frame)
clip = mpy.ImageSequenceClip( imageslist , fps = 25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration( clip.duration )
clip = clip.set_audio(audioclip=audio )
clip.write_videofile('part2_video.mp4', codec='libx264')

(360, 926, 3)


chunk:   0%|                                 | 0/159 [00:00<?, ?it/s, now=None]

Moviepy - Building video part2_video.mp4.
MoviePy - Writing audio in part2_videoTEMP_MPY_wvf_snd.mp3


t:   0%|                                     | 0/181 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part2_video.mp4



Moviepy - Done !
Moviepy - video ready part2_video.mp4


# PART3

In [29]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from matplotlib import pyplot as plt

def darker(pix,g):
    if (pix-g) <0 :
        return 0
    else:
        return (pix-g)
def darker_f_g_channel(pix,g): # at first the aim of this func was to keep green screen
    if (pix >= 235):
        return pix
    elif (pix-g) < 0:
        return 0
    else:
        return (pix-g)

g=90 # paremater for brightness
darker_vec = np.vectorize(darker)
darker_g_vec = np.vectorize(darker_f_g_channel)

background = cv2.imread('Malibu.jpg')
background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background, (int(background_width*ratio),360))
print(background.shape)

imageslist = []
i = 0
while 1:
    image=cv2.imread('cat/cat_'+ str(i) + '.png')
    if image is None:
        break
    i = i+1
    
    mirror_im = cv2.flip(image,1)
    green_im = np.zeros((360,286,3), np.uint8)
    green_im[:,0:286]= (0,255,0)
    mirror_im = cv2.hconcat([green_im, mirror_im])
    
    mirror_g_channel = mirror_im[:, :, 1]
    mirror_r_channel = mirror_im[:, :, 0]
    mirror_b_channel = mirror_im[:, :, 2]
    #vectorization to apply darker transform function
    darker_cat_g = darker_g_vec(mirror_g_channel,g)
    darker_cat_r = darker_vec(mirror_r_channel,g)
    darker_cat_b = darker_vec(mirror_b_channel,g)
    
    darker_cat = np.dstack((darker_cat_r,darker_cat_g,darker_cat_b))
    #only use darker cat values in frames
    with_myself = mirror_im.copy()
    foreground = np.logical_or(mirror_g_channel < 180 , mirror_r_channel > 150)
    n_x,n_y = np.nonzero(foreground)
    darker_cat_values = darker_cat[n_x, n_y, :]
    with_myself[n_x,n_y, : ] = darker_cat_values
    #and the left part, bright cat
    with_myself[0:image.shape[0],0:image.shape[1]-110,:] = image[0:image.shape[0],0:image.shape[1]-110,:]

    image = with_myself
    #same for the rest
    image_g_channel = image[:, :,1]
    image_r_channel = image[:, :,0]
    foreground = np.logical_or(image_g_channel < 180 , image_r_channel > 150)
    nonzero_x , nonzero_y = np.nonzero(foreground)
    nonzerocatvalues = image[nonzero_x, nonzero_y, : ]
    new_frame = background.copy()
    new_frame[nonzero_x, nonzero_y, :] = nonzerocatvalues
    new_frame = new_frame[:,:,[2, 1, 0]]
    imageslist.append(new_frame)
clip = mpy.ImageSequenceClip( imageslist , fps = 25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration( clip.duration )
clip = clip.set_audio(audioclip=audio )
clip.write_videofile('part3_video.mp4', codec='libx264')

(360, 926, 3)


chunk:   0%|                                 | 0/159 [00:00<?, ?it/s, now=None]

Moviepy - Building video part3_video.mp4.
MoviePy - Writing audio in part3_videoTEMP_MPY_wvf_snd.mp3


t:   0%|                                     | 0/181 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part3_video.mp4



Moviepy - Done !
Moviepy - video ready part3_video.mp4


# PART4

In [34]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from matplotlib import pyplot as plt

def calculate_cdf(hist):
    cdf = hist.cumsum()
    #cdf_normalized = cdf * hist.max()/ cdf.max() 
    cdf_normalized = cdf / float(cdf.max())
    return cdf_normalized

def lookup(source_cdf, target_cdf):
    lookup_table = np.zeros(256)
    lookup_val = 0
    for g_I in range(len(source_cdf)):
        for g_J in range(len(target_cdf)):
            if target_cdf[g_J] >= source_cdf[g_I]:
                lookup_val = g_J
                break
        lookup_table[g_I] = lookup_val
    return lookup_table

background = cv2.imread('Malibu.jpg')
background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background, (int(background_width*ratio),360))
print(background.shape)

target = cv2.imread('blue.jpg')

b_h_list = []
g_h_list = []
r_h_list = []
i = 0
while 1:
    image=cv2.imread('cat/cat_'+ str(i) + '.png')
    if image is None:
        break
    i = i+1
    
    mirror_im = cv2.flip(image,1)
    green_im = np.zeros((360,286,3), np.uint8)
    green_im[:,0:286]= (0,255,0)
    mirror_im = cv2.hconcat([green_im, mirror_im])
    
    mirror_g_channel = mirror_im[:, :, 1]
    mirror_r_channel = mirror_im[:, :, 2]
    mirror_b_channel = mirror_im[:, :, 0]
    
    mirror_hist_blue, bin_0 = np.histogram(mirror_b_channel.flatten(), 256, [0,256])
    mirror_hist_green, bin_1 = np.histogram(mirror_g_channel.flatten(), 256, [0,256])
    mirror_hist_red, bin_2 = np.histogram(mirror_r_channel.flatten(), 256, [0,256]) 
    
    b_h_list.append(mirror_hist_blue)
    g_h_list.append(mirror_hist_green)
    r_h_list.append(mirror_hist_red)

total_b = np.zeros_like(mirror_hist_blue)
total_g = np.zeros_like(mirror_hist_green)
total_r = np.zeros_like(mirror_hist_red)
for a in b_h_list:
    total_b = total_b + a
for a in g_h_list:
    total_g = total_g + a
for a in r_h_list:
    total_r = total_r + a

#average cat histogram:
average_b_histogram = total_b/i
average_g_histogram = total_g/i
average_r_histogram = total_r/i

target_g_channel = target[:, :, 1]
target_r_channel = target[:, :, 2]
target_b_channel = target[:, :, 0]

target_hist_blue, bin_3 = np.histogram(target_b_channel.flatten(), 256, [0,256])    
target_hist_green, bin_4 = np.histogram(target_g_channel.flatten(), 256, [0,256])
target_hist_red, bin_5 = np.histogram(target_r_channel.flatten(), 256, [0,256])

source_cdf_blue = calculate_cdf(average_b_histogram)
source_cdf_green = calculate_cdf(average_g_histogram)
source_cdf_red = calculate_cdf(average_r_histogram)
target_cdf_blue = calculate_cdf(target_hist_blue)
target_cdf_green = calculate_cdf(target_hist_green)
target_cdf_red = calculate_cdf(target_hist_red)

blue_lookup_table = lookup(source_cdf_blue, target_cdf_blue)
green_lookup_table = lookup(source_cdf_green, target_cdf_green)
red_lookup_table = lookup(source_cdf_red, target_cdf_red)

imageslist = []
i = 0
while 1:
    image=cv2.imread('cat/cat_'+ str(i) + '.png')
    if image is None:
        break
    i = i+1
    
    mirror_im = cv2.flip(image,1)
    green_im = np.zeros((360,286,3), np.uint8)
    green_im[:,0:286]= (0,255,0)
    mirror_im = cv2.hconcat([green_im, mirror_im])
    
    mirror_g_channel = mirror_im[:, :, 1]
    mirror_r_channel = mirror_im[:, :, 2]
    mirror_b_channel = mirror_im[:, :, 0]
    
    b_after_transform = cv2.LUT(mirror_b_channel, blue_lookup_table)
    g_after_transform = cv2.LUT(mirror_g_channel, green_lookup_table)
    r_after_transform = cv2.LUT(mirror_r_channel, red_lookup_table)
    
    image_after_matching = cv2.merge([b_after_transform, g_after_transform, r_after_transform])
    image_after_matching = cv2.convertScaleAbs(image_after_matching)
    
    with_myself = mirror_im.copy()
    foreground = np.logical_or(mirror_g_channel < 180 , mirror_r_channel > 150)
    n_x,n_y = np.nonzero(foreground)
    purple_cat_values = image_after_matching[n_x, n_y, :]
    with_myself[n_x,n_y, : ] = purple_cat_values
    with_myself[0:image.shape[0],0:image.shape[1]-110,:] = image[0:image.shape[0],0:image.shape[1]-110,:]
 
    image = with_myself
    #same for the rest
    image_g_channel = image[:, :,1]
    image_r_channel = image[:, :,2]
    foreground = np.logical_or(image_g_channel < 180 , image_r_channel > 150)
    nonzero_x , nonzero_y = np.nonzero(foreground)
    nonzerocatvalues = image[nonzero_x, nonzero_y, : ]
    new_frame = background.copy()
    new_frame[nonzero_x, nonzero_y, :] = nonzerocatvalues
    new_frame = new_frame[:,:,[2, 1, 0]]
    imageslist.append(new_frame)
clip = mpy.ImageSequenceClip( imageslist , fps = 25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration( clip.duration )
clip = clip.set_audio(audioclip=audio )
clip.write_videofile('part4_video.mp4', codec='libx264')

(360, 926, 3)


chunk:   0%|                                 | 0/159 [00:00<?, ?it/s, now=None]

Moviepy - Building video part4_video.mp4.
MoviePy - Writing audio in part4_videoTEMP_MPY_wvf_snd.mp3


t:   0%|                                     | 0/181 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part4_video.mp4



Moviepy - Done !
Moviepy - video ready part4_video.mp4
